## Can CO structure classification resolve the distance ambiguity for CO clouds near G309?

Look at cloud catalogs from two recent studies using slightly different techniques to de-construct Dame+ CfA/Columbia 12CO(J1-0) emission.

* Rice et al. (2016ApJ...822...52R): https://doi.org/10.3847/0004-637X/822/1/52
* Miville-Deschênes et al. (2017ApJ...834...57M): https://doi.org/10.3847/1538-4357/834/1/57

For Rice et al., an associated (preliminary) talk: http://www.mpia.de/HHSF14/proceedings/talks/tom_rice.pdf

From the CO survey decomposition into clouds of some geometric shape and average line emission, one can attempt kinematic distance resolution using an empirical size-linewidth relation (Larson's law).  For the cloud's averaged linewidth measurement and two possible sizes (near or far), favor the size that places the cloud's (linewidth,size) closer to the Larson relation.  A larger cloud will sample a larger line-of-sight velocity range, and may have larger turbulent / substructure broadening (but, what sets line-widths?  measurements/simulations probably offer insight).

Compare to this more manual (albeit more usefully interpreted for this project's needs) decomposition:

* García et al. (2014ApJS..212....2G): https://doi.org/10.1088/0067-0049/212/1/2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import astropy as ap
from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

from astropy.io import fits

import aplpy
from matplotlib.patches import Circle, Ellipse, Rectangle  # Scatter is inadequate for representing circles/ellipses

## Miville-Deschênes+ (2017) cloud catalog

The Miville-Deschênes table doesn't parse well.  Here is the API for table reading:
http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read

In [ ]:
#cols = ['Cloud', 'Ncomp', 'Npix', 'A', 'l', 'e_l', 'b', 'e_b', 'theta', 'WCO',
#        'NH2', 'Sigma', 'vcent', 'sigmav', 'Rmax', 'Rmin', 'Rang', 'Rgal',
#        'INF', 'Dn', 'Df', 'zn', 'zf', 'Sn', 'Sf', 'Rn', 'Rf', 'Mn', 'Mf']
#units=['', '', '', 'deg2', 'deg', 'deg', 'deg', 'deg', 'deg',
#       '', 'cm-2', 'solMass/pc2', 'km/s', 'km/s', 'deg', 'deg', 'deg', 'deg',
#       '', 'kpc', 'kpc', 'kpc', 'kpc', 'pc2', 'pc2', 'pc', 'pc', 'solMass', 'solMass']

t = Table.read('data/apjaa4dfdt1_mrt_miville-deschenes.txt', format='ascii', data_start=45,
              names=['Cloud', 'Ncomp', 'Npix', 'A', 'l', 'e_l', 'b', 'e_b', 'theta',
                     'WCO', 'NH2', 'Sigma', 'vcent', 'sigmav', 'Rmax', 'Rmin', 'Rang', 'Rgal',
                     'INF', 'Dn', 'Df', 'zn', 'zf', 'Sn', 'Sf', 'Rn', 'Rf', 'Mn', 'Mf'])

In [ ]:
# Sanity check column parsing
plt.subplot(1, 2, 1)
plt.hist(t['l']); plt.xlabel('l (deg.)');
plt.subplot(1, 2, 2)
plt.hist(t['b']); plt.xlabel('b (deg.)');
plt.tight_layout()

In [ ]:
mask = np.logical_and(abs(t['l'] % 360 - 309.2) - t['Rmax'] < 0.5,  # any part of cloud within 0.5 deg. of SNR
                      abs(t['b'] - -0.6) - t['Rmax'] < 0.5)  # any part of cloud within within 0.5 deg. of SNR
mask = np.logical_and(mask,
                      t['vcent'] - t['sigmav'] < -30)  # cloud velocity spread overlaps cen arm velocities
t[mask]

In [ ]:
fig = plt.figure(figsize=(14,10))

#fitsfig = aplpy.FITSFigure('data_intgr/submoment0_Th_IV_309.75_12CO.fits', figure=fig)
#fitsfig.show_colorscale(vmin=0, vmax=150000*0.55, stretch='linear',cmap='afmhot')

fitsfig = aplpy.FITSFigure('data_intgr/moment0_DHT36_Quad4_interp.fits', figure=fig)
fitsfig.show_colorscale(vmin=0, vmax=150, stretch='linear',cmap='afmhot')
fitsfig.recenter(309.2, -0.5, width=3, height=2)

# Is the -1* for angle correct?
# Ellipses are too small by a factor 1/2.
fitsfig.show_ellipses(t[mask]['l']%360, t[mask]['b'], t[mask]['Rmax'], t[mask]['Rmin'], angle=-1*t[mask]['theta'],
                      alpha=0.4, facecolor='b', edgecolor='g')

most = fits.open('../most/G309.2-0.6.fits')
lev = np.logspace(-2, -0.5, 4)  # Sparse log contours, 0.01 to 0.1*sqrt(10)
fitsfig.show_contour(data=most, levels=lev, colors='green', alpha=1)

fitsfig.refresh()

Perform a sanity check on the APLPy plotting wrappers - are angles and positions correct?

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.gca()

print "# Region file format: DS9 version 4.1"
print "global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1"
print "galactic"

for row in t[mask]:
    
    ellipse = Ellipse((row['l']%360, row['b']), row['Rmax'], row['Rmin'], row['theta'],
                      alpha=0.8 * row['NH2']/max(t[mask]['NH2']),  # NH2 propto W_CO / Npx
                      edgecolor='k')
    ax.add_patch(ellipse)

    c = SkyCoord(row['l']%360 * u.degree, row['b'] * u.degree, frame=ap.coordinates.Galactic)

    if row['INF'] == 0:
        d = row['Dn']
        fmt = ''
    else:
        d = row['Df']
        fmt = "color=blue width=2"
    
    # d in kpc, v in km/s
    print "ellipse({},{},{}\",{}\",{}) # {} text={{d={:.1f},v={:.1f}+/-{:.1f},npx={},WCO={:.1f}e3}}".format(
        c.galactic.l.to_string(sep=':', alwayssign=True, pad=True),
        c.galactic.b.to_string(sep=':', alwayssign=True, pad=True),
        row['Rmax'] * 3600 / 2,  # ARBITRARY DOWNSIZE FOR READABILITY...
        row['Rmin'] * 3600 / 2,
        -1*row['theta'],
        fmt,  # Use formatting to distinguish near/far
        d, row['vcent'], row['sigmav'], row['Npix'], row['WCO']/1e3)

ax.add_patch(Rectangle((309.0, -1), 1.4, 2, fill=False, edgecolor='r'))  # 309.0 to 310.4; -1 to 1

ax.set_ylim(-1.5, 1)
ax.set_xlim(308,310.4)
plt.xlabel('l (deg.)')
plt.ylabel('b (deg.)')
plt.gca().invert_xaxis()

## Rice+ (2016) cloud catalog

In [ ]:
r = Table.read('data/apj523087t3_mrt_rice.txt', format='ascii')

In [ ]:
plt.subplot(1, 2, 1)
plt.hist((r['GLON'] + 180) % 360 - 180); plt.xlabel('l (deg.)');
plt.subplot(1, 2, 2)
plt.hist(r['GLAT']); plt.xlabel('b (deg.)');
plt.tight_layout()

In [ ]:
mask = np.logical_and(abs(r['GLON'] - 309.2) - r['{sigma}r'] < 1,
                      abs(r['GLAT'] - -0.6) - r['{sigma}r'] < 1)
mask = np.logical_and(mask,
                      r['vLSR'] < 0)
r[mask]

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.gca()

print "# Region file format: DS9 version 4.1"
print "global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1"
print "galactic"

for row in r[mask]:
    
    circ = Circle((row['GLON'], row['GLAT']), row['{sigma}r'], facecolor='k', alpha=0.5, fill=True)
    ax.add_patch(circ)
    
    c = SkyCoord(row['GLON'] * u.degree, row['GLAT'] * u.degree, frame=ap.coordinates.Galactic)
    
    print "circle({},{},{}\") # text={{KDA={}, D={:.2f} kpc, {:.1f}+/-{:.1f} km/s}} color=magenta".format(
        c.galactic.l.to_string(sep=':', alwayssign=True, pad=True),
        c.galactic.b.to_string(sep=':', alwayssign=True, pad=True),
        row['{sigma}r'] * 3600,  # angular size
        row['KDA'], row['Distance'], row['vLSR'], row['{sigma}v'])

ax.set_ylim(-1.5, 1)
ax.set_xlim(308,310.4)
plt.xlabel('l (deg.)')
plt.ylabel('b (deg.)')
plt.gca().invert_xaxis()

In [ ]:
fig = plt.figure(figsize=(14,10))

#fitsfig = aplpy.FITSFigure('data_intgr/submoment0_Th_IV_309.75_12CO.fits', figure=fig)
#fitsfig.show_colorscale(vmin=0, vmax=150000*0.55, stretch='linear',cmap='afmhot')

fitsfig = aplpy.FITSFigure('data_intgr/moment0_DHT36_Quad4_interp.fits', figure=fig)
fitsfig.show_colorscale(vmin=0, vmax=150, stretch='linear',cmap='afmhot')
fitsfig.recenter(309.2, -0.5, width=3, height=2)

for row in r[mask]:
    fitsfig.show_circles(row['GLON'], row['GLAT'], row['{sigma}r'], facecolor='b', alpha=0.5)

most = fits.open('../most/G309.2-0.6.fits')
lev = np.logspace(-2, -0.5, 4)  # Sparse log contours, 0.01 to 0.1*sqrt(10)
fitsfig.show_contour(data=most, levels=lev, colors='green', alpha=1)

fitsfig.refresh()